In [1]:
import numpy as np
from scipy.stats import mode
import scipy
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from warnings import filterwarnings
filterwarnings('ignore')
import os
import cv2

In [2]:
import theano
import pymc3 as pm
import lasagne
from lasagne.utils import floatX
from lasagne.layers import InputLayer, DenseLayer, NonlinearityLayer, DropoutLayer
from lasagne.layers import Pool2DLayer as PoolLayer
from lasagne.layers import Conv2DLayer as ConvLayer
from lasagne.nonlinearities import rectify, softmax
import theano.tensor as T

floatX = theano.config.floatX

WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: GRID K520 (CNMeM is disabled, cuDNN 5005)


In [3]:
import sklearn
from sklearn import datasets
from sklearn.preprocessing import scale
from sklearn.cross_validation import train_test_split
from sklearn.datasets import fetch_mldata
from sklearn.metrics import accuracy_score

In [4]:
def preprocess(img, h=32, w=32):
    img = img.astype('float32')
    
    img_resized = np.zeros(shape=(1, 3, h, w))
    img_resized[0, 2, :, :] = cv2.resize(img[:, :, 0] - MEAN_VALUES[2], (h, w))
    img_resized[0, 1, :, :] = cv2.resize(img[:, :, 1] - MEAN_VALUES[1], (h, w)) 
    img_resized[0, 0, :, :] = cv2.resize(img[:, :, 2] - MEAN_VALUES[0], (h, w)) 
    
    return img_resized.astype('float32')

In [6]:
dir_path = 'data/train_subset/'

file_names = os.listdir(dir_path)
n_objects = len(file_names)

y_train = []
X_train = np.zeros(shape=(n_objects, 3, 32, 32), dtype='float32')

MEAN_VALUES = [0, 0, 0]
for i, f in enumerate(file_names):
    img = scipy.misc.imread(dir_path + f)
    y_train.append(f[:3] == 'cat')
    MEAN_VALUES[2] += img[:, :, 0].mean()
    MEAN_VALUES[1] += img[:, :, 1].mean()
    MEAN_VALUES[0] += img[:, :, 2].mean()

MEAN_VALUES = 1. * np.array(MEAN_VALUES) / n_objects
y_train = np.array(y_train, dtype='int32')

for i, f in enumerate(file_names):
    img = scipy.misc.imread(dir_path + f)
    X_train[i] = preprocess(img)

In [7]:
input_shape = list(X_train.shape)
input_shape[0] = None
input_shape = tuple(input_shape)
input_shape

(None, 3, 32, 32)

In [8]:
class PriorWeights(object):
    def __init__(self, mode='W', prior='gauss', **params):
        self.count = 0
        self.prior = prior
        self.mode = mode
        params.setdefault('std', 1.)
        params.setdefault('hyper', None)
        params.setdefault('layer', '')
        params.setdefault('pretrained', False)
        self.params = params
    def __call__(self, shape):
        self.count += 1
        if self.params['hyper'] is None:
            std = self.params['std']
        elif self.params['hyper'] == 'cauchy':
            std = pm.HalfCauchy('hyper_%s%d' % (self.mode, self.count), beta=1.)
        elif self.params['hyper'] == 'normal':
            std = pm.HalfNormal('hyper_%s%d' % (self.mode, self.count), mu=0., sd=1.)
        elif self.params['hyper'] == 'flat':
            std = pm.Flat()
            
        if self.params['pretrained']:
            tv = self.params['testval']
        else:
            tv = np.random.normal(size=shape).astype(np.float64)
        
        if self.prior == 'gauss':
            return pm.Normal('%s%s%d' % (self.params['layer'], self.mode, self.count), mu=tv, sd=std, 
                         testval=np.random.normal(size=shape).astype(np.float64), shape=shape)
        elif self.prior == 'laplace': 
            return pm.Laplace('%s%s%d' % (self.params['layer'], self.mode, self.count), mu=tv, b=std, 
                         testval=np.random.normal(size=shape).astype(np.float64), shape=shape)
        elif self.prior == 'flat':
            return pm.Flat('%s%s%d' % (self.params['layer'], self.mode, self.count), 
                           testval=np.random.normal(size=shape).astype(np.float64), shape=shape)

In [9]:
params_keys = ['conv1_1_W', 'conv1_1_b', 'conv1_2_W', 'conv1_2_b', 
            'conv2_1_W', 'conv2_1_b', 'conv2_2_W', 'conv2_2_b',
            'conv3_1_W', 'conv3_1_b', 'conv3_2_W', 'conv3_2_b', 'conv3_3_W', 'conv3_3_b', 'conv3_4_W', 'conv3_4_b',
            'conv4_1_W', 'conv4_1_b', 'conv4_2_W', 'conv4_2_b', 'conv4_3_W', 'conv4_3_b', 'conv4_4_W', 'conv4_4_b',
            'conv5_1_W', 'conv5_1_b', 'conv5_2_W', 'conv5_2_b', 'conv5_3_W', 'conv5_3_b', 'conv5_4_W', 'conv5_4_b',
            'fc6_W', 'fc6_b']#, 'fc7_W', 'fc7_b', 'fc8_W', 'fc8_b']

params = np.load('vgg19.npz')['params']

for i in range(32, len(params)): 
    params[i] = params[i].T

In [10]:
params[0].shape

(64, 3, 3, 3)

In [11]:
priors = {}
for i, param_key in enumerate(params_keys):
    priors[param_key] = PriorWeights(mode=param_key[-1], prior='gauss', pretrained=True, 
                                     layer=param_key[:-1], testval=params[i])

priors['fc6_W'] = PriorWeights(mode='W', prior='gauss')
priors['fc6_b'] = PriorWeights(mode='b', prior='gauss')

In [12]:
#vgg19 model
#http://www.robots.ox.ac.uk/~vgg/research/very_deep/
def build_ann(priors, input_var, target_var, 
              input_shape):

    with pm.Model() as neural_network:
        net = {}
        net['input'] = InputLayer((None, 3, 64, 64), input_var=input_var)
        net['conv1_1'] = ConvLayer(net['input'], 64, 3, pad=1,
                                   W=priors['conv1_1_W'], b=priors['conv1_1_b'],
                                   flip_filters=False)
        net['conv1_2'] = ConvLayer(net['conv1_1'], 64, 3, pad=1,
                                   W=priors['conv1_2_W'], b=priors['conv1_2_b'],
                                   flip_filters=False)
        net['pool1'] = PoolLayer(net['conv1_2'], 2)
        net['conv2_1'] = ConvLayer(net['pool1'], 128, 3, pad=1,
                                   W=priors['conv2_1_W'], b=priors['conv2_1_b'],
                                   flip_filters=False)
        net['conv2_2'] = ConvLayer(net['conv2_1'], 128, 3, pad=1,
                                   W=priors['conv2_2_W'], b=priors['conv2_2_b'],
                                   flip_filters=False)
        net['pool2'] = PoolLayer(net['conv2_2'], 2)
        net['conv3_1'] = ConvLayer(net['pool2'], 256, 3, pad=1,
                                   W=priors['conv3_1_W'], b=priors['conv3_1_b'],
                                   flip_filters=False)
        net['conv3_2'] = ConvLayer(net['conv3_1'], 256, 3, pad=1,
                                   W=priors['conv3_2_W'], b=priors['conv3_2_b'],
                                   flip_filters=False)
        net['conv3_3'] = ConvLayer(net['conv3_2'], 256, 3, pad=1,
                                   W=priors['conv3_3_W'], b=priors['conv3_3_b'],
                                   flip_filters=False)
        net['conv3_4'] = ConvLayer(net['conv3_3'], 256, 3, pad=1,
                                   W=priors['conv3_4_W'], b=priors['conv3_4_b'],
                                   flip_filters=False)
        net['pool3'] = PoolLayer(net['conv3_4'], 2)
        net['conv4_1'] = ConvLayer(net['pool3'], 512, 3, pad=1,
                                   W=priors['conv4_1_W'], b=priors['conv4_1_b'],
                                   flip_filters=False)
        net['conv4_2'] = ConvLayer(net['conv4_1'], 512, 3, pad=1,
                                   W=priors['conv4_2_W'], b=priors['conv4_2_b'],
                                   flip_filters=False)
        net['conv4_3'] = ConvLayer(net['conv4_2'], 512, 3, pad=1,
                                   W=priors['conv4_3_W'], b=priors['conv4_3_b'],
                                   flip_filters=False)
        net['conv4_4'] = ConvLayer(net['conv4_3'], 512, 3, pad=1,
                                   W=priors['conv4_4_W'], b=priors['conv4_4_b'],
                                   flip_filters=False)
        net['pool4'] = PoolLayer(net['conv4_4'], 2)
        net['conv5_1'] = ConvLayer(net['pool4'], 512, 3, pad=1,
                                   W=priors['conv5_1_W'], b=priors['conv5_1_b'],
                                   flip_filters=False)
        net['conv5_2'] = ConvLayer(net['conv5_1'], 512, 3, pad=1,
                                   W=priors['conv5_2_W'], b=priors['conv5_2_b'],
                                   flip_filters=False)
        net['conv5_3'] = ConvLayer(net['conv5_2'], 512, 3, pad=1,
                                   W=priors['conv5_3_W'], b=priors['conv5_3_b'],
                                   flip_filters=False)
        net['conv5_4'] = ConvLayer(net['conv5_3'], 512, 3, pad=1,
                                   W=priors['conv5_4_W'], b=priors['conv5_4_b'],
                                   flip_filters=False)
        net['pool5'] = PoolLayer(net['conv5_4'], 2)
        net['fc6'] = DenseLayer(net['pool5'],
                                   W=priors['fc6_W'], b=priors['fc6_b'],
                                   num_units=2)
#         net['fc6_dropout'] = DropoutLayer(net['fc6'], p=0.5)
#         net['fc7'] = DenseLayer(net['fc6_dropout'], 
#                                    W=priors['fc7_W'], b=priors['fc7_b'],
#                                    num_units=4096)
#         net['fc7_dropout'] = DropoutLayer(net['fc7'], p=0.5)
#         net['fc8'] = DenseLayer(net['fc7_dropout'], 
#                                    W=priors['fc8_W'], b=priors['fc8_b'],
#                                    num_units=2, nonlinearity=None)
        net['prob'] = NonlinearityLayer(net['fc6'], softmax)
        
        prediction = lasagne.layers.get_output(net['prob'])
        
        out = pm.Categorical('out', prediction,
                           observed=target_var,
                           total_size=y_train.shape[0])
        
    return neural_network

In [13]:
def create_minibatch(data):
    rng = np.random.RandomState(0)
    
    while True:
        ixs = rng.randint(len(data), size=100)
        yield data[ixs]

In [14]:
def get_prediction(samples_proba):
    return mode(np.argmax(sample_proba(X_test, 500), 
                          axis=-1), axis=0).mode[0]

In [ ]:
minibatch_X = pm.generator(create_minibatch(X_train))
minibatch_y = pm.generator(create_minibatch(y_train))
neural_network_minibatch = build_ann(priors, minibatch_X, minibatch_y, input_shape)

In [ ]:
ADVI_ITERS = 5000
N_SAMPLES = 500

In [ ]:
with neural_network_minibatch:
    inference = pm.ADVI()
    approx = pm.fit(ADVI_ITERS, method=inference)